# 第3章: 正規表現

> Wikipediaの記事を以下のフォーマットで書き出したファイル[jawiki-country.json.gz](https://nlp100.github.io/data/jawiki-country.json.gz)がある．  
> - 1行に1記事の情報がJSON形式で格納される  
> - 各行には記事名が”title”キーに，記事本文が”text”キーの辞書オブジェクトに格納され，そのオブジェクトがJSON形式で書き出される  
> - ファイル全体はgzipで圧縮される
>
> 以下の処理を行うプログラムを作成せよ．


解答は基本Qiitaの[【言語処理100本ノック 2020】第3章: 正規表現](https://qiita.com/yamaru/items/255d0c5dcb2d1d4ccc14)を利用

In [ ]:
import os
import json
import re
import requests
from pprint import pprint

In [ ]:
# データを保存するディレクトリの作成
DATADIR = "data"
CURRENTDIR = "/workspace/notebook"
CHAPDIR = os.path.join(DATADIR, "chapter3")

try:
    os.mkdir(CHAPDIR)
except:
    print("作成済み等の理由でディレクトリが作成されませんでした")

In [ ]:
# 利用するファイルの取得
!wget -P $CURRENTDIR/$CHAPDIR "https://nlp100.github.io/data/jawiki-country.json.gz"

# 解凍
!gunzip $CURRENTDIR/$CHAPDIR/jawiki-country.json.gz

In [ ]:
# 利用するデータファイル名
in_filename = "jawiki-country.json"

## 20. JSONデータの読み込み

>Wikipedia記事のJSONファイルを読み込み，「イギリス」に関する記事本文を表示せよ.問題21-29では，ここで抽出した記事本文に対して実行せよ．

In [ ]:
in_fpath = os.path.join(CHAPDIR, in_filename)

with open(in_fpath, "r", encoding="utf8")as f:
    # 1行1レコード
    for line in f:
        record = json.loads(line)
        if record["title"] == "イギリス":
            text_uk = record["text"]
            break
            
print(text_uk)

## 21. カテゴリ名を含む行を抽出

> 記事中でカテゴリ名を宣言している行を抽出せよ．

In [ ]:
pattern = r"^(.*\[\[Category:.*\]\].*)$"
result = re.findall(pattern, text_uk, re.MULTILINE)
pprint(result)

# 22. カテゴリ名の抽出

> 記事のカテゴリ名を（行単位ではなく名前で）抽出せよ．

In [ ]:
pattern = r"^.*\[\[Category:(.*?)(?:\|.*)?\]\].*$"
result = re.findall(pattern, text_uk, re.MULTILINE)
pprint(result)

## 23. セクション構造

> 記事中に含まれるセクション名とそのレベル（例えば”== セクション名 ==”なら1）を表示せよ．

In [ ]:
pattern = r"^(\={2,})\s*(.+?)\s*(\={2,}).*$"
result = [i[1] + ":" + str(len(i[0]) - 1) for i in re.findall(pattern, text_uk, re.MULTILINE)]
pprint(result)

## 24. ファイル参照の抽出

> 記事から参照されているメディアファイルをすべて抜き出せ．

In [ ]:
pattern = r"\[\[ファイル:(.+?)\|"
result = re.findall(pattern, text_uk)
pprint(result)

## 25. テンプレートの抽出

> 記事中に含まれる「基礎情報」テンプレートのフィールド名と値を抽出し，辞書オブジェクトとして格納せよ．

In [ ]:
pattern = r"^\{\{基礎情報.*?$(.*?)^\}\}"
template = re.findall(pattern, text_uk, re.MULTILINE + re.DOTALL)
print(template)


pattern = r"^\|(.+?)\s*=\s*(.+?)(?:(?=\n\|)|(?=\n$))"
result = dict(re.findall(pattern, template[0], re.MULTILINE + re.DOTALL))
for k, v in result.items():
    print(f"{k}: {v}")

## 26. 強調マークアップの除去

> 25の処理時に，テンプレートの値からMediaWikiの強調マークアップ（弱い強調，強調，強い強調のすべて）を除去してテキストに変換せよ（参考: マークアップ早見表）．

In [ ]:
### 解答
def remove_markup(text):
  # 強調マークアップの除去
    pattern = r"'{2,5}"
    text = re.sub(pattern, "", text)

    return text

result_rm = {k: remove_markup(v) for k, v in result.items()}
for k, v in result_rm.items():
    print(f"{k}: {v}")

## 27. 内部リンクの除去

> 26の処理に加えて，テンプレートの値からMediaWikiの内部リンクマークアップを除去し，テキストに変換せよ（参考: マークアップ早見表）．

In [ ]:
### 解答
def remove_markup(text):
    # 強調マークアップの除去
    pattern = r"\"{2,5}"
    text = re.sub(pattern, "", text)
    pattern = r"\[\[(?:[^|]*?\|)??([^|]*?)\]\]"
    text = re.sub(pattern, r"\1", text)

    return text

result_rm = {k: remove_markup(v) for k, v in result.items()}
for k, v in result_rm.items():
    print(f"{k}: {v}")

## 28. MediaWikiマークアップの除去

> 27の処理に加えて，テンプレートの値からMediaWikiマークアップを可能な限り除去し，国の基本情報を整形せよ．

In [ ]:
### 解答
def remove_markup(text):
    # 強調マークアップの除去
    pattern = r"\"{2,5}"
    text = re.sub(pattern, "", text)

    # 内部リンクマークアップの除去
    pattern = r"\[\[(?:[^|]*?\|)??([^|]*?)\]\]"
    text = re.sub(pattern, r"\1", text)

    # 外部リンクマークアップの除去
    # これはおそらくURLの正規表現の一般系? -> 参考URL：https://qiita.com/str32/items/a692073af32757618042
    pattern = r"https?://[\w!?/\+\-_~=;\.,*&@#$%\(\)\'\[\]]+"
    text = re.sub(pattern, "", text)

    # htmlタグの除去
    pattern = r"<.+?>" 
    text = re.sub(pattern, "", text)

    # テンプレートの除去
    pattern = r"\{\{(?:lang|仮リンク)(?:[^|]*?\|)*?([^|]*?)\}\}" 
    text = re.sub(pattern, r"\1", text)

    return text

result_rm = {k: remove_markup(v) for k, v in result.items()}
for k, v in result_rm.items():
    print(f"{k}: {v}")

# 29. 国旗画像のURLを取得する

## テンプレートの内容を利用し，国旗画像のURLを取得せよ．（ヒント: MediaWiki APIのimageinfoを呼び出して，ファイル参照をURLに変換すればよい）

In [ ]:
### 解答


def get_url(text):
    url_file = text["国旗画像"].replace(" ", "_")
    url = "https://commons.wikimedia.org/w/api.php?action=query&titles=File:" + url_file + "&prop=imageinfo&iiprop=url&format=json"
    data = requests.get(url)
    return re.search(r'"url":"(.+?)"', data.text).group(1)

print(get_url(result))